In [332]:
import pandas as pd
import matplotlib.pyplot as plt

In [334]:
df1 = pd.read_csv("../../data/WellProduction_subset.csv",parse_dates=['ProdPeriod'],index_col=False)
df1 = df1.drop(columns=['WellHeader.Match','Unnamed: 0'])

In [335]:
df1.head()

,EPAssetsId,ProdPeriod,ProdType,Volume
0,2588946,2019-12-31,Production Hours,744.0
1,2608797,2019-12-31,Production Hours,744.0
2,2617557,2019-12-31,Production Hours,744.0
3,2588149,2019-12-31,Production Hours,744.0
4,2583451,2019-12-31,Production Hours,744.0


In [336]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99337 entries, 0 to 99336
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   EPAssetsId  99337 non-null  int64         
 1   ProdPeriod  99337 non-null  datetime64[ns]
 2   ProdType    99337 non-null  object        
 3   Volume      99337 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 3.0+ MB


## Create data frame for each Type of production monthly

In [337]:
## Group the Volume per Prod Type for each month in the data seet and fill with 0 for NaNa
df2=df1.groupby(['EPAssetsId','ProdPeriod','ProdType']).sum().unstack('ProdPeriod')
#df2.fillna(value=0,inplace = True) Will fill NaNa to the very end so the mean it is calculated correctly

In [338]:
df2.head()

Volume                          \
ProdPeriod                              2015-01-31 2015-02-28   2015-03-31   
EPAssetsId ProdType                                                          
2225574    Gas Production (MMcf)               NaN        NaN     6.222468   
           Oil Production (Bbls)               NaN        NaN  5027.437140   
           Production Hours                    NaN        NaN   150.000000   
           Water Production (Bbls)             NaN        NaN          NaN   
2575232    Condensate Production (Bbls)        NaN  357.26064          NaN   

                                                                   \
ProdPeriod                                2015-04-30   2015-05-31   
EPAssetsId ProdType                                                 
2225574    Gas Production (MMcf)            5.615053     7.892858   
           Oil Production (Bbls)         3049.924020  5304.188340   
           Production Hours               467.000000   740.000000   
           Water Production (Bbls)               NaN  7938.985560   
2575232    Condensate Production (Bbls)          NaN          NaN   

                                                                     \
ProdPeriod                                 2015-06-30    2015-07-31   
EPAssetsId ProdType                                                   
2225574    Gas Production (MMcf)             5.717466      3.107702   
           Oil Production (Bbls)          4863.902340   3936.156840   
           Production Hours                408.000000    314.000000   
           Water Production (Bbls)       10258.663800  11344.283280   
2575232    Condensate Production (Bbls)           NaN           NaN   

                                                                     \
ProdPeriod                                 2015-08-31    2015-09-30   
EPAssetsId ProdType                                                   
2225574    Gas Production (MMcf)             3.082982      4.919352   
           Oil Production (Bbls)          4031.761800   4925.542380   
           Production Hours                358.000000    501.000000   
           Water Production (Bbls)       10184.444160  10153.624140   
2575232    Condensate Production (Bbls)           NaN    490.604400   

                                                      ...               \
ProdPeriod                                2015-10-31  ...   2019-04-30   
EPAssetsId ProdType                                   ...                
2225574    Gas Production (MMcf)            4.555609  ...     0.720422   
           Oil Production (Bbls)         3896.531100  ...   462.300300   
           Production Hours               486.000000  ...   576.000000   
           Water Production (Bbls)       9086.874060  ...  5062.660020   
2575232    Condensate Production (Bbls)   689.991060  ...          NaN   

                                                                               \
ProdPeriod                                2019-05-31  2019-06-30   2019-07-31   
EPAssetsId ProdType                                                             
2225574    Gas Production (MMcf)            0.734548     0.21542     0.134196   
           Oil Production (Bbls)          259.139760   130.82784   184.291140   
           Production Hours               480.000000   384.00000   668.000000   
           Water Production (Bbls)       2461.827720  2644.86090  4231.777440   
2575232    Condensate Production (Bbls)          NaN         NaN          NaN   

                                                                               \
ProdPeriod                               2019-08-31   2019-09-30   2019-10-31   
EPAssetsId ProdType                                                             
2225574    Gas Production (MMcf)            1.00294     1.709236     2.185986   
           Oil Production (Bbls)          630.23796   782.451120   854.783820   
           Production Hours               744.00000   720.000000   744.000000   


In [339]:
#Reset index so the prodType is now a column
df3 = df2.reset_index('ProdType')

In [340]:
# identify the prodType values
prod_type=df3['ProdType'].unique()
prod_type

array(['Gas Production (MMcf)', 'Oil Production (Bbls)',
       'Production Hours', 'Water Production (Bbls)',
       'Condensate Production (Bbls)'], dtype=object)

In [341]:

def create_labels(prodtype,df):
    """ Create new labels for columns with the type of produciton and date"""
    months =['January','February','March','April','May','June','July','August','September','October','November','December']
    years = ['2015','2016','2017','2018','2019','2020']
    n =len(df.columns)
    labels =[]

    while n > 0:
        for year in years:
                for month in months:
                    if n >0:
                        labels.append(prodtype+" "+ month+" " + year)
                        n = n-1
    df.columns=labels                
    return df

In [342]:
# create a dataframe list with a dataframe for each prodType
dfs=[]
for product in prod_type:
    #filter by type
    df = df3[df3['ProdType']== product]
    #change from multindex columns to a single index
    df.columns = df.columns.droplevel()
    df = df.drop(columns="")
    df =create_labels(product, df)
    #calcualte total volume and monthly volume.
    total = df.sum(axis=1,skipna=True)
    mean = df.mean(axis=1,skipna=True)
    df[product+' Total'] = total
    df[product + ' Average'] = mean
    dfs.append(df.reset_index())


In [343]:
#checking number of wells in dataframes
for df in dfs:
    print(df.shape)

(1135, 64)
(947, 64)
(1109, 64)
(727, 64)
(176, 64)


Just 176 wells have condensate production, and 727 water production

In [344]:
#assing dataframe names
gas_production = dfs[0]
oil_production = dfs[1]
time_production = dfs[2]
water_production = dfs[3]
condensate_production = dfs[4]

In [345]:
#check the dataframe
gas_production.sample(10)

,EPAssetsId,Gas Production (MMcf) January 2015,Gas Production (MMcf) February 2015,Gas Production (MMcf) March 2015,Gas Production (MMcf) April 2015,Gas Production (MMcf) May 2015,Gas Production (MMcf) June 2015,Gas Production (MMcf) July 2015,Gas Production (MMcf) August 2015,Gas Production (MMcf) September 2015,...,Gas Production (MMcf) June 2019,Gas Production (MMcf) July 2019,Gas Production (MMcf) August 2019,Gas Production (MMcf) September 2019,Gas Production (MMcf) October 2019,Gas Production (MMcf) November 2019,Gas Production (MMcf) December 2019,Gas Production (MMcf) January 2020,Gas Production (MMcf) Total,Gas Production (MMcf) Average
302,2595518,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.127133,0.183637,0.067098,0.077693,0.077693,0.635666,0.264861,0.321365,25.882217,0.681111
45,2583433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.031192,0.812240,0.882870,0.854618,0.519128,1.214829,1.020598,1.190109,58.912149,1.636449
63,2583484,NaN,NaN,NaN,1.571509,4.209524,5.092394,2.620358,3.425536,5.102989,...,2.390812,2.267210,2.302525,2.041195,2.246021,2.051790,1.635075,1.702173,197.646341,3.467480
520,2605856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.516172,4.527357,3.951726,4.961729,5.011170,4.859316,4.855785,4.721589,154.255046,5.932886
292,2595365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.024129,2.464973,2.281336,2.055321,1.825775,1.811649,1.882279,1.748083,112.996766,2.824919
573,2608347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.029403,6.829882,7.497332,7.038240,5.862257,6.120055,4.163615,0.752205,227.932314,9.117293
344,2598046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.815134,19.942268,19.433734,18.406074,18.172996,16.710963,17.403133,16.972293,1126.789324,30.453766
297,2595483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.015293,4.181272,4.177741,4.064733,3.782215,3.605641,3.743369,3.517354,228.094762,6.164723
888,2616408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.154794,1.412592,1.006472,1.108885,0.586226,0.695702,0.695702,0.695702,10.117690,0.843141
1119,2622840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,15.489071,59.473655,54.925108,129.887834,43.295945


## Join dataframes 

In [346]:
join1=pd.merge(gas_produciton, water_production, on='EPAssetsId', how='outer').fillna(0)

In [347]:
join2=pd.merge(join1,oil_production, on='EPAssetsId', how='outer').fillna(0)
join3=pd.merge(join2,time_production, on ='EPAssetsId', how='outer').fillna(0)
final_df=pd.merge(join3,condensate_production, on ='EPAssetsId', how='outer').fillna(0)

In [348]:
final_df.tail()

,EPAssetsId,Gas Production (MMcf) January 2015,Gas Production (MMcf) February 2015,Gas Production (MMcf) March 2015,Gas Production (MMcf) April 2015,Gas Production (MMcf) May 2015,Gas Production (MMcf) June 2015,Gas Production (MMcf) July 2015,Gas Production (MMcf) August 2015,Gas Production (MMcf) September 2015,...,Condensate Production (Bbls) June 2019,Condensate Production (Bbls) July 2019,Condensate Production (Bbls) August 2019,Condensate Production (Bbls) September 2019,Condensate Production (Bbls) October 2019,Condensate Production (Bbls) November 2019,Condensate Production (Bbls) December 2019,Condensate Production (Bbls) January 2020,Condensate Production (Bbls) Total,Condensate Production (Bbls) Average
1130,2624145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1131,2624210,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1132,2624211,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1133,2624624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1134,2625908,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [349]:
final_df.shape

(1135, 314)

In [350]:
#safe file
final_df.to_csv('production1.csv',index=False)

# Plots

In [352]:
df1.head()

,EPAssetsId,ProdPeriod,ProdType,Volume
0,2588946,2019-12-31,Production Hours,744.0
1,2608797,2019-12-31,Production Hours,744.0
2,2617557,2019-12-31,Production Hours,744.0
3,2588149,2019-12-31,Production Hours,744.0
4,2583451,2019-12-31,Production Hours,744.0
